In [ ]:
from scannerpy import Client, DeviceType
from scannerpy.storage import NamedVideoStream, NamedStream
from query.models import Video

from esper.table_tennis.utils import *
from esper.widget import *

import os
import sys
import numpy as np

In [ ]:
movie_path = '/app/data/videos/sample-clip.mp4'
movie_name = os.path.splitext(os.path.basename(movie_path))[0]
video = Video.objects.filter(path__contains='sample')[0]

In [ ]:
sc = Client()
stride = 1
input_stream = NamedVideoStream(sc, movie_name, path=movie_path)
frame = sc.io.Input([input_stream])
strided_frame = sc.streams.Stride(frame, [stride])

# MaskRCNN

In [ ]:
from scannertools import maskrcnn_detection

object_frame = sc.ops.MaskRCNNDetectObjects(
    frame=strided_frame,
    device=DeviceType.GPU, #if sc.has_gpu() else DeviceType.CPU,
    batch=8,
    )

In [ ]:
maskrcnn_stream = NamedStream(sc, movie_name + '_maskrcnn')

In [ ]:
output_op = sc.io.Output(object_frame, [maskrcnn_stream])
sc.run(output_op, scannerpy.common.PerfParams.estimate(), cache_mode=scannerpy.CacheMode.Overwrite)

In [ ]:
fid = 100
seq = sc.sequence(maskrcnn_stream._name)
obj = seq.load(workers=1, row=[fid])

In [ ]:
metadata = next(obj)
frame = load_frame(video, fid, [])
maskrcnn_detection.visualize_labels(frame, metadata, min_score_thresh=0.7)
imshow(frame)

# DensePose

In [ ]:
from scannertools import densepose_detection
densepose_frame = sc.ops.DensePoseDetectPerson(
    frame=strided_frame,
    device=DeviceType.GPU, #if sc.has_gpu() else DeviceType.CPU,
    batch=1
    )

In [ ]:
densepose_stream = NamedStream(sc, movie_name + '_densepose')

In [ ]:
output_op = sc.io.Output(object_frame, [densepose_stream])
sc.run(output_op, scannerpy.common.PerfParams.estimate(), cache_mode=scannerpy.CacheMode.Overwrite)

In [ ]:
fid = 100
seq = sc.sequence(densepose_stream._name)
obj = seq.load(workers=1, rows=[fid])
metadata = next(obj)

In [ ]:
frame = load_frame(video, fid, [])
image = densepose_detection.visualize_one_image(frame, metadata, min_score_thresh=0.7, show_keypoint=True, show_box=True)
imshow(image)

In [ ]:
frame = load_frame(video, fid, [])
image = densepose_detection.visualize_uvbody(frame, metadata, min_score_thresh=0.7)
imshow(image)

In [ ]:
video = Video.objects.filter(path__contains='men_single')[1]
video.path